In [2]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

In [33]:
df = pd.read_csv("src/data/fooddataset.csv", encoding='latin-1')
df.head()

,Unnamed: 0,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1961,1961,1000 persons,8954.0,NaN
1,1,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1962,1962,1000 persons,9142.0,NaN
2,2,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1963,1963,1000 persons,9340.0,NaN
3,3,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1964,1964,1000 persons,9547.0,NaN
4,4,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1965,1965,1000 persons,9765.0,NaN


In [30]:
df = pd.read_csv("src/data/FoodBalanceSheetsHistoric_E_All_Data.csv", encoding='latin-1')
df.head()


,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F
0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1000 persons,8954.00,NaN,9142.00,...,27708.00,NaN,28398.00,NaN,29105.00,NaN,29825.00,NaN,30552.00,NaN
1,2,Afghanistan,2901,Grand Total,664,Food supply (kcal/capita/day),kcal/capita/day,2999.00,Fc,2917.00,...,2081.00,Fc,2104.00,Fc,2107.00,Fc,2100.00,Fc,2090.00,Fc
2,2,Afghanistan,2901,Grand Total,674,Protein supply quantity (g/capita/day),g/capita/day,84.91,Fc,82.98,...,57.79,Fc,58.14,Fc,58.91,Fc,58.91,Fc,58.25,Fc
3,2,Afghanistan,2901,Grand Total,684,Fat supply quantity (g/capita/day),g/capita/day,37.51,Fc,37.61,...,30.72,Fc,33.88,Fc,33.08,Fc,33.37,Fc,33.52,Fc
4,2,Afghanistan,2903,Vegetal Products,664,Food supply (kcal/capita/day),kcal/capita/day,2752.00,Fc,2672.00,...,1871.00,Fc,1888.00,Fc,1891.00,Fc,1883.00,Fc,1873.00,Fc


In [43]:
a = df.pivot(index="Year", columns=["Value", "Area", "Item Code", "Element Code"])

: 

: 

In [34]:
mozambique = df[df.Area == 'Mozambique']

grouped_mo = mozambique.groupby('Element').sum()
grouped_mo.loc['Combined'] = grouped_mo.loc['Feed']+grouped_mo.loc['Food']
grouped_mo.loc['Prop_Food'] = grouped_mo.loc['Food']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo.loc['Prop_Feed'] = grouped_mo.loc['Feed']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo=grouped_mo.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()
# grouped_mo = grouped_mo.iloc[[5,19,20]]
# grouped_mo
grouped_mo
# melted_mo = pd.melt(grouped_mo.iloc[18:19], id_vars=['Element'], value_vars=grouped_mo.columns)
# melted_mo['variable'] = melted_mo.variable.str.lstrip('Y')

,Element,Unnamed: 0,Year Code,Year,Value
0,Domestic supply quantity,3.188744e+10,1.298906e+07,1.298906e+07,1.190547e+06
1,Export Quantity,2.496805e+10,1.025985e+07,1.025985e+07,4.654363e+04
2,Fat supply quantity (g/capita/day),2.681099e+10,1.109326e+07,1.109326e+07,8.025050e+03
3,Feed,3.376623e+09,1.804603e+06,1.804603e+06,6.587734e+04
4,Food,3.037084e+10,1.231742e+07,1.231742e+07,8.068430e+05
5,Food supply (kcal/capita/day),3.150349e+10,1.274676e+07,1.274676e+07,4.514880e+05
6,Food supply quantity (kg/capita/yr),3.037112e+10,1.231742e+07,1.231742e+07,5.054463e+04
7,Import Quantity,2.986670e+10,1.219341e+07,1.219341e+07,1.080790e+05
8,Losses,1.099414e+10,4.658382e+06,4.658382e+06,5.771900e+04
9,Other uses (non-food),8.116311e+09,3.498312e+06,3.498312e+06,6.029560e+03


In [17]:
import json

f = open("src/data/world3.json")
geojson = json.load(f)


In [25]:
df

,Unnamed: 0,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1961,1961,1000 persons,8954.00,NaN
1,1,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1962,1962,1000 persons,9142.00,NaN
2,2,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1963,1963,1000 persons,9340.00,NaN
3,3,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1964,1964,1000 persons,9547.00,NaN
4,4,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1965,1965,1000 persons,9765.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13533226,2975292,5817,Net Food Importing Developing Countries,2899,Miscellaneous,684,Fat supply quantity (g/capita/day),2015,2015,g/capita/day,0.01,Fc
13533227,2975293,5817,Net Food Importing Developing Countries,2899,Miscellaneous,684,Fat supply quantity (g/capita/day),2016,2016,g/capita/day,0.01,Fc
13533228,2975294,5817,Net Food Importing Developing Countries,2899,Miscellaneous,684,Fat supply quantity (g/capita/day),2017,2017,g/capita/day,0.01,Fc
13533229,2975295,5817,Net Food Importing Developing Countries,2899,Miscellaneous,684,Fat supply quantity (g/capita/day),2018,2018,g/capita/day,0.01,Fc


In [18]:
area_replacements = [
  ("United Kingdom of Great Britain and Northern Ireland", "United Kingdom"), 
  ("United States of America", "United States"),
  ("Czechia", "Czech Republic"),
  ("Republic of Moldova", "Moldova"),
  ("North Macedonia", "Macedonia"),
  ("Iran (Islamic Republic of)", "Iran"),
  ("Congo", "Democratic Republic of Congo"),
  ("Venezuela (Bolivarian Republic of)", "Venezuela"),
  ("Bolivia (Plurinational State of)", "Bolivia"),
  ("Viet Nam", "Vietnam")
]

for pair in area_replacements:
  df.Area = df.Area.replace(pair[0], pair[1])

In [19]:
a = df[(df.Item == "Grand Total") & (df.Element == "Food supply (kcal/capita/day)")]

for feature in geojson['features']:
  try: 
    area = feature['properties']['NAME_LONG']
    mozambique = df[df.Area == area]

    grouped_mo = mozambique.groupby('Element').sum()
    grouped_mo.loc['Combined'] = grouped_mo.loc['Feed']+grouped_mo.loc['Food']
    grouped_mo.loc['Prop_Food'] = grouped_mo.loc['Food']/grouped_mo.loc['Total Population - Both sexes']
    grouped_mo.loc['Prop_Feed'] = grouped_mo.loc['Feed']/grouped_mo.loc['Total Population - Both sexes']
    grouped_mo=grouped_mo.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()

    ids = [3, 4, 19, 20]
    labels = ["feed", "food", "prop_food", "prop_feed"]

    for idx, label in enumerate(labels):
      feature['properties'][label] = list(grouped_mo.iloc[idx][1:])

    feature['properties']['food_supply'] = list(a[a.Area == area].groupby("Element").sum().values[0][3:])
  except KeyError:
    print("Country not found: ", area)
  
  

Country not found:  Aland Islands
Country not found:  Andorra
Country not found:  French Southern and Antarctic Lands
Country not found:  Bahrain
Country not found:  Bhutan
Country not found:  Côte d'Ivoire
Country not found:  Cyprus U.N. Buffer Zone
Country not found:  Republic of Congo
Country not found:  Cape Verde
Country not found:  Northern Cyprus
Country not found:  Eritrea
Country not found:  Dhekelia
Country not found:  Falkland Islands
Country not found:  Faeroe Islands
Country not found:  The Gambia
Country not found:  Equatorial Guinea
Country not found:  Greenland
Country not found:  Hong Kong
Country not found:  Isle of Man
Country not found:  Baikonur Cosmodrome
Country not found:  Siachen Glacier
Country not found:  Kosovo
Country not found:  Lao PDR
Country not found:  Puerto Rico
Country not found:  Dem. Rep. Korea
Country not found:  Palestine
Country not found:  Qatar
Country not found:  Western Sahara
Country not found:  South Sudan
Country not found:  Singapore
Co

In [23]:
out = open("src/data/out.json", "w+")
json.dump(geojson, out)

In [24]:

mozambique = df[df.Area == "Mozambique"]

grouped_mo = mozambique.groupby('Element').sum()
grouped_mo.loc['Combined'] = grouped_mo.loc['Feed']+grouped_mo.loc['Food']
grouped_mo.loc['Prop_Food'] = grouped_mo.loc['Food']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo.loc['Prop_Feed'] = grouped_mo.loc['Feed']/grouped_mo.loc['Total Population - Both sexes']
grouped_mo=grouped_mo.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()
# grouped_mo = grouped_mo.iloc[[3, 45,17,18]]
grouped_mo
# list(grouped_mo.iloc[3])

,Element,Unnamed: 0,Year Code,Year,Value
0,Domestic supply quantity,3.188744e+10,1.298906e+07,1.298906e+07,1.190547e+06
1,Export Quantity,2.496805e+10,1.025985e+07,1.025985e+07,4.654363e+04
2,Fat supply quantity (g/capita/day),2.681099e+10,1.109326e+07,1.109326e+07,8.025050e+03
3,Feed,3.376623e+09,1.804603e+06,1.804603e+06,6.587734e+04
4,Food,3.037084e+10,1.231742e+07,1.231742e+07,8.068430e+05
5,Food supply (kcal/capita/day),3.150349e+10,1.274676e+07,1.274676e+07,4.514880e+05
6,Food supply quantity (kg/capita/yr),3.037112e+10,1.231742e+07,1.231742e+07,5.054463e+04
7,Import Quantity,2.986670e+10,1.219341e+07,1.219341e+07,1.080790e+05
8,Losses,1.099414e+10,4.658382e+06,4.658382e+06,5.771900e+04
9,Other uses (non-food),8.116311e+09,3.498312e+06,3.498312e+06,6.029560e+03


In [21]:
grouped = df.groupby('Element').sum()
grouped.loc['Combined'] = grouped.loc['Feed']+grouped.loc['Food']
grouped.loc['Prop_Food'] = grouped.loc['Food']/grouped.loc['Combined']
grouped.loc['Prop_Feed'] = grouped.loc['Feed']/grouped.loc['Combined']
grouped=grouped.drop(['Area Code', 'Item Code', 'Element Code'], axis = 1).reset_index()
grouped.iloc[4].mean

<bound method NDFrame._add_numeric_operations.<locals>.mean of Element           Food
Y1961      14167404.24
Y1962      14553648.68
Y1963      14800692.71
Y1964      15151075.18
Y1965      15648214.11
Y1966      16137188.34
Y1967      16518483.22
Y1968       16926715.6
Y1969      17390735.63
Y1970      17910556.04
Y1971      18124543.13
Y1972      18266472.68
Y1973      19023930.27
Y1974      19389058.76
Y1975      19818062.48
Y1976       20062780.1
Y1977      20626304.94
Y1978      21380490.43
Y1979      21733056.35
Y1980      21989721.49
Y1981      22431743.63
Y1982      23070481.05
Y1983       23722190.3
Y1984      24342005.66
Y1985      24795958.19
Y1986      25406894.64
Y1987      26069954.16
Y1988      26600181.56
Y1989      27175995.69
Y1990      27562072.73
Y1991      27852822.28
Y1992      28769965.54
Y1993      29516838.37
Y1994      30324685.08
Y1995      31346712.09
Y1996      32132718.01
Y1997      32950610.58
Y1998      33860697.77
Y1999      35060176.27
Y2000       36028